In [1]:
import pandas as pd
import numpy as np

COLUMNS_NAME = ['nome_squadra', 'numero_cella_x', 'numero_cella_y', 'frequenza_eventi', 'frequenza_passaggi_accurati',
                'frequenza_tiri', 'frequenza_falli_subiti']


# HELP FUNCTION
def make_list(list_dict, key='id'):
    if len(list_dict) == 0:
        return []
    return [i[key] for i in list_dict]


def get_x(lista):
    if not lista:
        return None
    return lista[0]['x']


def get_y(lista):
    if not lista:
        return None
    return lista[0]['y']

input_folder = 'C:/Users/Davide/workspace/SoccerData/Input/'

# IMPORT DATA
data = pd.read_json(input_folder + 'events.json')
players = pd.read_json(input_folder + 'players.json')
data = data[data.playerId != 0]

In [2]:
data.shape

(600585, 12)

In [3]:
data.head(1)

eventId eventName  eventSec         id  matchId matchPeriod  playerId  \
0        8      Pass  2.530536  180423957  2575959          1H      8327   

                                  positions subEventId subEventName  \
0  [{'y': 52, 'x': 49}, {'y': 44, 'x': 43}]         85  Simple pass   

             tags  teamId  
0  [{'id': 1801}]    3158

In [4]:
#df = pd.read_csv('dataset.csv', index_col=0)
players = pd.read_json(input_folder +'players.json')
events = pd.read_json(input_folder +'events.json')
matches = pd.read_json(input_folder +'matches.json')
teams = pd.read_json(input_folder +'teams.json')

squadre = {}

for name in teams.name.values:
    squadre[name] = teams[teams.name==name].wyId.values[0]

squadre['Hellas'] = squadre['Hellas Verona']
squadre.pop('Hellas Verona')
    
def get_home(label):
    i=0
    temp = ''
    while label[i] != ' ':
        temp = temp + label[i]
        i= i + 1
    return temp
    
dic = {}
lab_match = matches.label.values
for label in lab_match:
    home = get_home(label)
    team_id = squadre[home]
    match_id = matches[matches.label == label].wyId.values[0]
    dic[match_id] = team_id


def make_list(list_dict, key='id'):
    if len(list_dict) == 0:
        return []
    return [i[key] for i in list_dict]


def get_x(lista):
    if not lista:
        return None
    return lista[0]['x']


def get_y(lista):
    if not lista:
        return None
    return lista[0]['y']

events.tags = events.tags.apply(make_list)

df = events[['playerId', 'matchId', 'matchPeriod', 'teamId', 'eventId', 'eventName',
             'subEventId', 'subEventId', 'tags']]
# FROM POSITIONS TO SINGLE VALUE COLUMNS x,y
df['x'] = events.positions.apply(get_x)
df['y'] = events.positions.apply(get_y)

df.head(10)

df['home_team'] = df.matchId.map(dic)

df.loc[(df.teamId != df.home_team) & (df.matchPeriod=='1H'), 'x'] = 100 - df.loc[(df.teamId != df.home_team) &
                                                                                    (df.matchPeriod=='1H'), 'x']
df.loc[(df.teamId == df.home_team) & (df.matchPeriod=='2H'), 'x'] = 100 - df.loc[(df.teamId == df.home_team) & 
                                                                                    (df.matchPeriod=='2H'), 'x']
df.loc[(df.teamId != df.home_team) & (df.matchPeriod=='1H'), 'y'] = 100 - df.loc[(df.teamId != df.home_team) &
                                                                                    (df.matchPeriod=='1H'), 'y']
df.loc[(df.teamId == df.home_team) & (df.matchPeriod=='2H'), 'y'] = 100 - df.loc[(df.teamId == df.home_team) & 
                                                                                    (df.matchPeriod=='2H'), 'y']


C:\Users\Davide\Anaconda3\lib\site-packages\ipykernel\__main__.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Davide\Anaconda3\lib\site-packages\ipykernel\__main__.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [8]:
df.head()

playerId  matchId matchPeriod  teamId  eventId           eventName  \
0      8327  2575959          1H    3158        8                Pass   
1     20438  2575959          1H    3158        8                Pass   
2      8306  2575959          1H    3158        7  Others on the ball   
3      8306  2575959          1H    3158        1                Duel   
4      8306  2575959          1H    3158        1                Duel   

  subEventId subEventId              tags   x   y  home_team  
0         85         85            [1801]  51  48       3172  
1         85         85            [1801]  57  56       3172  
2         72         72                []  64  83       3172  
3         11         11  [504, 703, 1801]  22  44       3172  
4         11         11  [503, 703, 1801]  36  85       3172

In [9]:
# FROM FIELD POSITIONS TO GRID
bins = np.arange(0, 120, 20)
label = [0, 1, 2, 3, 4]
df['numero_cella_x'] = pd.cut(df.x, bins=bins, labels=label).fillna(0.).astype(int)
df['numero_cella_y'] = pd.cut(df.y, bins=bins, labels=label).fillna(0.).astype(int)

In [10]:
df.head()

playerId  matchId matchPeriod  teamId  eventId           eventName  \
0      8327  2575959          1H    3158        8                Pass   
1     20438  2575959          1H    3158        8                Pass   
2      8306  2575959          1H    3158        7  Others on the ball   
3      8306  2575959          1H    3158        1                Duel   
4      8306  2575959          1H    3158        1                Duel   

  subEventId subEventId              tags   x   y  home_team  numero_cella_x  \
0         85         85            [1801]  51  48       3172               2   
1         85         85            [1801]  57  56       3172               2   
2         72         72                []  64  83       3172               3   
3         11         11  [504, 703, 1801]  22  44       3172               1   
4         11         11  [503, 703, 1801]  36  85       3172               1   

   numero_cella_y  
0               2  
1               2  
2               4  
3               2  
4               4

In [12]:
res = []
for ix,i in data.groupby(['matchId','matchPeriod','playerId']):
    res.append(event_by_cell(i,ix))

In [15]:
def event_by_cell(df,ix):
    tt = df.groupby(['numero_cella_x','numero_cella_y']).size().reset_index()
    tt.index = [str(i) + str(j) for i, j in zip(tt.numero_cella_x, tt.numero_cella_y)]
    tt = tt[0]
    tt['matchId'] = ix[0]
    tt['matchPeriod'] = ix[1] 
    #tt['playerId'] = ix[2]
    return tt

In [16]:
tot = []
for ix,i in data.groupby(['matchId','matchPeriod']):
    tot.append(event_by_cell(i,ix))

In [18]:
t= pd.concat(tot,1).T.fillna(0)
t

00  01  02  03  04  10  11  12  13  14     ...       32  33  34  40  41  \
0   22  16  16  13  12  55  34  28  50  53     ...       20  33  42  18  11   
0   19  22  32  17  25  49  43  39  36  37     ...       26  49  43  37  11   
0   24  10  28  15   7  54  54  33  53  40     ...       26  44  59   9   8   
0   17  15  29  17  11  36  50  32  33  36     ...       27  47  37  19  14   
0   27  18  22  16   4  27  47  38  42  26     ...       28  27  23   5  11   
0   32  19  20  11  15  22  46  47  43  14     ...       55  42  31  14   8   
0   42  21  31  27  13  65  41  24  47  35     ...       18  33  35  14  18   
0   26  14  33  27  11  56  65  26  40  39     ...       24  30  42  19  18   
0   19  17  37  14  22  52  69  32  53  52     ...       22  42  51  14  12   
0   19  13  32  22  12  27  42  21  48  49     ...       36  52  58  19  12   
0   19  15  30  21  20  46  50  38  50  48     ...       22  43  43  23  17   
0    7  10  36  15  24  37  62  40  53  39     ...       20  32  32  29  13   
0   31  15  29  15  12  54  54  63  48  32     ...       38  41  42  22  15   
0   28  14  36  25  12  58  53  55  50  35     ...       48  49  51  19   5   
0   41  26  29  21  10  43  37  27  49  32     ...       12  34  55  25  15   
0   29  24  26  30  19  50  50  33  54  43     ...       16  18  32  33  22   
0   23  17  34  12  11  38  57  39  43  39     ...       19  38  38  14   8   
0   22  22  30  18  16  35  41  34  31  35     ...       32  38  33  21  15   
0   24  17  28  22  10  36  53  32  44  53     ...       26  43  54  27   9   
0   27  15  26  11   5  45  49  33  65  38     ...       31  45  61  15   6   
0   29  20  24  18  12  48  67  24  58  47     ...       10  30  52  22  16   
0   37  23  24  14   9  41  43  27  52  36     ...       12  30  32  19   8   
0   33  19  45  14  14  32  54  43  45  32     ...       30  27  29   7   5   
0   29  13  30  20  12  37  30  42  29  30     ...       27  37  30  29  11   
0   33  18  28  14  10  53  56  37  50  27     ...       20  35  35  12   8   
0   29  12  26  24  12  30  73  33  69  50     ...       28  35  32  13  11   
0   18  24  40  28   8  35  63  32  45  36     ...       18  34  28  14  15   
0   26  23  32  23  16  25  31  32  50  29     ...       28  35  21  17  13   
0   26  22  29  17  11  27  52  30  48  42     ...       28  37  25  13  15   
0   19  20  27  13  11  36  89  44  65  38     ...       28  34  37  12   7   
..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..     ...       ..  ..  ..  ..  ..   
0   29  14  28  25   9  64  26  53  33  35     ...       31  31  62  11  15   
0   20  13  32  19  10  41  48  32  28  25     ...       46  69  41  18  17   
0   34  25  49  24  16  48  41  32  41  65     ...       23  28  51  28  17   
0   25  29  57  24  42  60  47  28  45  54     ...       21  27  34  25  10   
0   22  16  32  20  19  44  55  36  44  40     ...       32  54  42  24  17   
0   24  13  46  25  23  47  36  39  44  37     ...       32  34  58  52  30   
0   30  19  16  11  20  44  31  32  39  53     ...       18  21  41  16  11   
0   31  31  27  30  17  43  37  21  35  36     ...       15  40  58  19  20   
0   27  17  23  12  16  26  52  44  34  38     ...       22  31  33  20   6   
0   29   9  20  19  14  26  20  27  29  36     ...       29  25  32  20   8   
0   22  21  42  15  22  38  57  48  47  51     ...       31  53  69  28   9   
0   19  19  32  15   9  38  38  30  38  53     ...       30  29  24  21  13   
0   24  15  15  10  15  45  66  36  63  45     ...       27  53  81  17   6   
0   31  24  27  11  13  44  44  34  65  37     ...       21  47  66  13   7   
0   17  13  30  22  13  37  69  36  48  43     ...       23  30  38  28  15   
0   24  11  34  15   9  27  37  30  48  42     ...       20  26  38  13  11   
0   14  15  37  20  13  43  47  39  60  37     ...       64  63  43  22  11   
0   14  14  40  29  17  53  58  49  67  25     ...       44  26  17  32  23   
0   20  22  26  22  12  55  52  48  43  55     ...    

In [38]:
data = pd.read_csv(input_folder + 'dataset.csv', index_col=0)
data.drop([i[:2] for i in x],1, inplace=True)

In [39]:
result = pd.concat([pd.DataFrame(dd[x].values/dd[y].values,columns=[i[:2] for i in x]), 
 dd[['matchId','matchPeriod','playerId']]],1)

C:\Users\Davide\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':


In [41]:
data.merge(result, on = ['matchId','matchPeriod','playerId']).to_csv('dataset.csv')

In [ ]:
data.

In [31]:
dd = r.merge(t, on = ['matchId','matchPeriod'])
x = ['00_x', '01_x', '02_x', '03_x', '04_x', '10_x', '11_x', '12_x', '13_x',
       '14_x', '20_x', '21_x', '22_x', '23_x', '24_x', '30_x', '31_x', '32_x',
       '33_x', '34_x', '40_x', '41_x', '42_x', '43_x', '44_x']

y = ['00_y', '01_y', '02_y', '03_y', '04_y',
       '10_y', '11_y', '12_y', '13_y', '14_y', '20_y', '21_y', '22_y', '23_y',
       '24_y', '30_y', '31_y', '32_y', '33_y', '34_y', '40_y', '41_y', '42_y',
       '43_y', '44_y']

dd

00_x  01_x  02_x  03_x  04_x  10_x  11_x  12_x  13_x  14_x  ...   30_y  \
0         1     0     0     0     0     0     0     1     6     3  ...     62   
1         1     1     0     0     0     7     2     0     0     0  ...     62   
2         4     1     1     0     0    10     0     1     0     0  ...     62   
3         0     0     0     0     0     2     2     1     0     1  ...     62   
4         0     4     1     0     1     0     0     2     9     4  ...     62   
5         0     1     0     2     0     1     3     1     4     2  ...     62   
6         3     1     0     0     0     9     5     1     0     0  ...     62   
7         2     3     1     0     0     9    10     3     0     0  ...     62   
8         0     0     0     0     0     2     1     0     3     0  ...     62   
9         0     0     0     0     1     3     0     0     0     0  ...     62   
10        0     0     0     0     0     1     0     1     3     2  ...     62   
11        0     0     0     0     0     0     0     0     1     5  ...     62   
12        0     0     1     2     3     2     0     0     7     5  ...     62   
13        0     0     0     5     3     0     0     0     1     9  ...     62   
14        2     1     1     0     0     5     4     3     0     0  ...     62   
15        5     0     2     2     0     0     0     0     0     0  ...     62   
16        0     2     0     0     0     0     5     9    10     4  ...     62   
17        0     0     0     1     2     0     0     4     6     2  ...     62   
18        1     2     6     1     0     0     0     0     0     0  ...     62   
19        0     0     1     0     0     0     0     1     0     4  ...     62   
20        0     0     1     0     2     0     0     0     0    12  ...     62   
21        3     0     1     0     0     4     2     0     0     0  ...     62   
22        1     1     0     0     0     2     0     0     3     1  ...     67   
23        0     0     0     0     0     0     1     4     0     0  ...     67   
24        0     0     0     1     0     0     0     3     0     0  ...     67   
25        0     0     0     0     0     1     0     0     0     1  ...     67   
26        3     5     1     0     0    15     2     0     0     0  ...     67   
27        0     0     0     0     0     0     0     1     2     0  ...     67   
28        0     0     0     0     3     0     0     1     7     3  ...     67   
29        0     0     0     0     0     4     0     0     0     0  ...     67   
...     ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...    ...   
18756     0     0     0     0     0     9     2     1     0     0  ...     43   
18757     0     0     2     0     1     0     0     0     1    10  ...     43   
18758     0     0     0     0     0     0     0     1     0     0  ...     43   
18759     1     1     1     0     1     2    10     2     0     1  ...     43   
18760     0     0     0     0     0     0     0     0     0     0  ...     26   
18761     0     0     0     0     0     0     1     1     1     0  ...     26   
18762     3     2     1     0     0     0     2     0     1     0  ...     26   
18763     0     0     0     1     0     3     3     5     5     0  ...     26   
18764     1     0     1     1     0     5     2     6     2     0  ...     26   
18765     0     0     0     0     0     1     2     1     1     0  ...     26   
18766     0     1     3     5     4     0     0     0     2     2  ...     26   
18767     0     0     0     0     0     0     2     0     0     0  ...     26   
18768     0     0     0     0     1     4     0     2     1     4  ...     26   
18769     2     5     0     0     0     1     3     0     0     0  ...     26   
18770     3     2     7     3     0     0     0     0     0     0  ...     26   
18771     0     1     1     0     0     3     2     1     4     2  ...     26   
18772     0     0     0     0     0     0     0     1     0     0  ...     26   
18773     1     2     8     1     6   

In [13]:
r= pd.concat(res,1).T.fillna(0)
r

00  01  02  03  04  10  11  12  13  14    ...     33  34  40  41  42  43  \
0    1   0   0   0   0   0   0   1   6   3    ...      4   6   0   1   0   1   
0    1   1   0   0   0   7   2   0   0   0    ...      1   0   4   1   0   1   
0    4   1   1   0   0  10   0   1   0   0    ...      1   0   1   0   0   1   
0    0   0   0   0   0   2   2   1   0   1    ...      6   0   1   3   0   1   
0    0   4   1   0   1   0   0   2   9   4    ...      1   8   0   0   1   0   
0    0   1   0   2   0   1   3   1   4   2    ...      3   0   0   0   0   0   
0    3   1   0   0   0   9   5   1   0   0    ...      0   0   0   0   0   0   
0    2   3   1   0   0   9  10   3   0   0    ...      0   0   0   0   0   0   
0    0   0   0   0   0   2   1   0   3   0    ...      2   1   0   0   1   2   
0    0   0   0   0   1   3   0   0   0   0    ...      0   0   6   1   0   1   
0    0   0   0   0   0   1   0   1   3   2    ...      3   3   0   1   0   0   
0    0   0   0   0   0   0   0   0   1   5    ...      4   5   0   0   0   1   
0    0   0   1   2   3   2   0   0   7   5    ...      2   4   0   0   0   0   
0    0   0   0   5   3   0   0   0   1   9    ...      2   8   0   0   0   0   
0    2   1   1   0   0   5   4   3   0   0    ...      0   0   2   0   0   0   
0    5   0   2   2   0   0   0   0   0   0    ...      0   0   0   0   0   0   
0    0   2   0   0   0   0   5   9  10   4    ...      0   0   0   0   0   0   
0    0   0   0   1   2   0   0   4   6   2    ...      0   0   0   0   0   0   
0    1   2   6   1   0   0   0   0   0   0    ...      0   0   0   0   0   0   
0    0   0   1   0   0   0   0   1   0   4    ...      2   2   2   3   0   0   
0    0   0   1   0   2   0   0   0   0  12    ...      2   5   0   0   0   2   
0    3   0   1   0   0   4   2   0   0   0    ...      0   0   2   1   0   0   
0    1   1   0   0   0   2   0   0   3   1    ...      5   2   0   0   1   1   
0    0   0   0   0   0   0   1   4   0   0    ...      2   3   0   0   0   2   
0    0   0   0   1   0   0   0   3   0   0    ...      0   0   0   0   0   0   
0    0   0   0   0   0   1   0   0   0   1    ...      0   2   4   1   0   2   
0    3   5   1   0   0  15   2   0   0   0    ...      0   0   4   0   0   0   
0    0   0   0   0   0   0   0   1   2   0    ...      5   0   0   1   2   2   
0    0   0   0   0   3   0   0   1   7   3    ...      5   0   0   0   0   0   
0    0   0   0   0   0   4   0   0   0   0    ...      0   0   0   1   0   0   
..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..    ...     ..  ..  ..  ..  ..  ..   
0    0   0   0   0   0   9   2   1   0   0    ...      0   1   4   1   0   0   
0    0   0   2   0   1   0   0   0   1  10    ...      2  15   0   0   1   4   
0    0   0   0   0   0   0   0   1   0   0    ...      1   0   0   0   0   1   
0    1   1   1   0   1   2  10   2   0   1    ...      0   0   0   0   0   0   
0    0   0   0   0   0   0   0   0   0   0    ...      7   0   0   3   2   3   
0    0   0   0   0   0   0   1   1   1   0    ...      1   3   0   0   0   1   
0    3   2   1   0   0   0   2   0   1   0    ...      0   0   0   0   0   0   
0    0   0   0   1   0   3   3   5   5   0    ...      0   1   0   0   0   0   
0    1   0   1   1   0   5   2   6   2   0    ...      2   4   0   0   0   0   
0    0   0   0   0   0   1   2   1   1   0    ...      0   4   0   0   0   0   
0    0   1   3   5   4   0   0   0   2   2    ...      1   4   0   0   0   0   
0    0   0   0   0   0   0   2   0   0   0    ...      3   2   5   4   2   0   
0    0   0   0   0   1   4   0   2   1   4    ...      2   0   0   0   4   1   
0    2   5   0   0   0   1   3   0   0   0    ...      0   0   0   0   0   0   
0    3   2   7   3   0   0   0   0   0   0    ...      0   0   0   0   0   0   
0    0   1   1   0   0   3   2   1   4   2    ...      2   1   0   0   1   0   
0    0   0   0   0   0   0   0   1   0   0    ...      1   0   2   0   0   1   
0    1   2   8   1   6   0   0   1   0   1    ...      0   0   0   0   0   0   
0   

In [11]:
data.groupby(['matchId','matchPeriod','playerId','numero_cella_x','numero_cella_y']).size().reset_index()

matchId matchPeriod  playerId  numero_cella_x  numero_cella_y   0
0       2575959          1H       114               0               0   1
1       2575959          1H       114               1               2   1
2       2575959          1H       114               1               3   6
3       2575959          1H       114               1               4   3
4       2575959          1H       114               2               1   1
5       2575959          1H       114               2               2   2
6       2575959          1H       114               2               3   2
7       2575959          1H       114               2               4   3
8       2575959          1H       114               3               0   3
9       2575959          1H       114               3               1   4
10      2575959          1H       114               3               2   2
11      2575959          1H       114               3               3   4
12      2575959          1H       114               3               4   6
13      2575959          1H       114               4               1   1
14      2575959          1H       114               4               3   1
15      2575959          1H       114               4               4   2
16      2575959          1H      3475               0               0   1
17      2575959          1H      3475               0               1   1
18      2575959          1H      3475               1               0   7
19      2575959          1H      3475               1               1   2
20      2575959          1H      3475               2               0   5
21      2575959          1H      3475               2               1   1
22      2575959          1H      3475               3               0   8
23      2575959          1H      3475               3               1   7
24      2575959          1H      3475               3               2   1
25      2575959          1H      3475               3               3   1
26      2575959          1H      3475               4               0   4
27      2575959          1H      3475               4               1   1
28      2575959          1H      3475               4               3   1
29      2575959          1H      8306               0               0   4
...         ...         ...       ...             ...             ...  ..
193516  2576338          2H    349102               2               3   9
193517  2576338          2H    349102               3               1   5
193518  2576338          2H    349102               3               2   8
193519  2576338          2H    349102               3               3   7
193520  2576338          2H    349102               3               4   8
193521  2576338          2H    349102               4               3   2
193522  2576338          2H    349102               4               4   3
193523  2576338          2H    354552               0               3   1
193524  2576338          2H    354552               0               4   5
193525  2576338          2H    354552               1               3   6
193526  2576338          2H    354552               1               4   4
193527  2576338          2H    354552               2               3   3
193528  2576338          2H    354552               2               4   2
193529  2576338          2H    354552               3               2   1
193530  2576338          2H    354552               3               3   4
193531  2576338          2H    354552               3               4   7
193532  2576338          2H    354552               4               3   1
193533  2576338          2H    354552               4               4   4
193534  2576338          2H    413041               0               0   5
193535  2576338          2H    413041               0               1   4
193536  2576338          2H    413041               0               2   2
193537  2576338          2H    413041               1               1   2
193538  2576

In [6]:
# EVENTI TOTALI
total_events_by_team = data.groupby(['matchId', 'teamId']).size().reset_index(name='total_events_cell')
total_events_by_cell = data.groupby(['numero_cella_x', 'numero_cella_y', 'teamId']).size().reset_index(
    name='number_events_cell')

df = total_events_by_cell.merge(total_events_by_team, on=['teamId'])
df['events_freq'] = df.number_events_cell / df.total_events_cell
df.drop(['number_events_cell', 'total_events_cell'], 1, inplace=True)


In [7]:
# PASSAGGI
accurate_pass = data.loc[[idx for idx, row in data.iterrows() if 1801 in row['tags'] and row.eventId == 8]]

In [8]:
accurate_pass_by_team = accurate_pass.groupby(['matchId', 'teamId']).size().reset_index(name='num_total_pass')
accurate_pass_by_cell = accurate_pass.groupby(['numero_cella_x', 'numero_cella_y', 'teamId']).size().reset_index(
    name='number_acc_pass')


In [9]:
df_pass = accurate_pass_by_cell.merge(accurate_pass_by_team, on=['teamId'])
df_pass['acc_pass_freq'] = df_pass.number_acc_pass / df_pass.num_total_pass

df = df.merge(df_pass[['numero_cella_x', 'numero_cella_y', 'teamId', 'acc_pass_freq']], how='left',
              on=['numero_cella_x', 'numero_cella_y', 'teamId'])


In [10]:
# TIRI
shot = data.loc[data.eventId == 10]


In [11]:
shot_by_team = shot.groupby(['matchId', 'teamId']).size().reset_index(name='num_total_shot')
shot_by_cell = shot.groupby(['numero_cella_x', 'numero_cella_y', 'teamId']).size().reset_index(name='number_shot')


In [12]:
df_shot = shot_by_cell.merge(shot_by_team, on=['teamId'])
df_shot['shot_freq'] = df_shot.number_shot / df_shot.num_total_shot

df = df.merge(df_shot[['numero_cella_x', 'numero_cella_y', 'teamId', 'shot_freq']], how='left',
              on=['numero_cella_x', 'numero_cella_y', 'teamId'])

In [13]:
# RESULTS PREPARATION
df = df[['matchId', 'teamId', 'numero_cella_x', 'numero_cella_y', 'events_freq', 'acc_pass_freq', 'shot_freq']]
df = df.fillna(0)

In [14]:
df.head()

matchId  teamId  numero_cella_x  numero_cella_y  events_freq  \
0  2575961    3157               0               0     0.851393   
1  2575961    3157               0               0     0.851393   
2  2575961    3157               0               0     0.851393   
3  2575961    3157               0               0     0.851393   
4  2575961    3157               0               0     0.851393   

   acc_pass_freq  shot_freq  
0       0.363934        0.0  
1       0.654867        0.0  
2       0.403636        0.0  
3       0.470339        0.0  
4       0.495536        0.0

In [15]:
df.to_csv('pos_su_griglia.csv')